In [2]:
from onnxruntime.training import artifacts
import torch
import onnx
import transformers

In [3]:
pipeline = transformers.pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
)

transformers_model = transformers.LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", config=transformers.LlamaConfig(num_hidden_layers=1), ignore_mismatched_sizes=True)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([32000, 2048]) in the checkpoint and torch.Size([32000, 4096]) in the model instantiated
- model.embed_tokens.weight: found shape torch.Size([32000, 2048]) in the checkpoint and torch.Size([32000, 4096]) in the model instantiated
- model.layers.0.input_layernorm.weight: found shape torch.Size([2048]) in the checkpoint and torch.Size([4096]) in the model instantiated
- model.layers.0.mlp.down_proj.weight: found shape torch.Size([2048, 5632]) in the checkpoint and torch.Size([4096, 11008]) in the model instantiated
- model.layers.0.mlp.gate_proj.weight: found shape torch.Size([5632, 2048]) in the checkpoint and torch.Size([11008, 4096]) in the model instantiated
- model.layers.0.mlp.up_proj.weight: found shape torch.Size([5632, 2048]) in the checkpoint and torch.Size([1100

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
inputs = tokenizer("The capital of France is Paris.", return_tensors="pt")

In [5]:
class FlatModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *local_inputs):
        return self.model(inputs.input_ids, inputs.attention_mask)

# model = FlatModel(pipeline.model)
model = FlatModel(transformers_model)

In [6]:
input_names = ["input_ids", "attention_mask", "position_ids"]
output_names = ["loss", "logits"]

torch.onnx.export(model,
                  (inputs["input_ids"], inputs["attention_mask"]),
                  "tinyllama.onnx",
                  input_names = input_names, 
                  output_names = output_names,
                  export_params=True,
                  training=torch.onnx.TrainingMode.TRAINING,
                  do_constant_folding=False,
                  dynamic_axes={
                    "input_ids": {0: "batch_size", 1: "sequence_length"},
                    "attention_mask": {0: "batch_size", 1: "sequence_length"},
                    "logits": {0: "batch_size", 1: "sequence_length"}
                  }
                  )

c:\Users\carolinezhu\AppData\Local\anaconda3\envs\generate-artifacts-3\lib\site-packages\transformers\models\llama\modeling_llama.py:1057: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_length > self.causal_mask.shape[-1]:


In [8]:
# requires_grad = []
# frozen_params = []
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         requires_grad.append(name)
#     else:
#         frozen_params.append(name)

# for name, param in model.named_buffers():
#     frozen_params.append(name)

onnx_model = onnx.load("tinyllama.onnx")

requires_grad = [param.name for param in onnx_model.graph.initializer]
frozen_params = []

artifacts.generate_artifacts(
    onnx_model,
    requires_grad=requires_grad,
    frozen_params=frozen_params,
    optimizer=artifacts.OptimType.AdamW,
    loss_input_names=["loss"]

)



hello hello


RuntimeError: C:\a\_work\1\s\orttraining\orttraining\python\orttraining_pybind_state.cc:617 onnxruntime::python::addObjectMethodsForTraining::<lambda_bb073955579d98a9752b4412a7cc4c49>::operator () [ONNXRuntimeError] : 1 : FAIL : Node (/model/lm_head/MatMul_Grad/Reshape_4) Op (Reshape) [ShapeInferenceError] Dimension could not be inferred: incompatible shapes
